In [1]:
from openai import OpenAI
import os
import pandas as pd
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List
from tqdm import tqdm
import json, csv
load_dotenv('/Users/guida/llm_argument_tasks/.env')

api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
class ArgumentClassification(BaseModel):
    id: str = Field(description="The ID the comment being analyzed")    
    label: int = Field(description="The label associated with the argument (0 or 1)") 

## Gay Marriage

In [42]:
def classify_text_gpt(id: str, comment_text: str, argument: str) -> dict:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
        {"role": "system", "content": f"""
        Analyze the given comment about gay marriage in relation to a specific argument. Your need to:
        Identify if the comment makes use the given argument. If it does, assign the label 1. If it does not, assign the label 0.
        Do NOT use any other label.
        Do NOT include the comment or the argument in the response.
        
        The argument to analyze is: {argument}
        
        Provide your response in the following JSON format:
        
        {{
            "id": "{id}",
            "label": "the label for the use of the argument in the comment"
        }}
        
        Analyze the following comment in relation to the given argument
        """},
        {"role": "user", "content": comment_text},
        ],
        response_format=ArgumentClassification,
    )

    return completion.choices[0].message.content

In [45]:
gm = pd.read_csv('../../clean_data/GM_structured.csv')

def process_comments_with_arguments(df: pd.DataFrame) -> List[dict]:
    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing comments"):
        comment_id = row['id'] 
        comment_text = row['comment_text']  
        argument_text = row['argument_text']

        try:
            classification = classify_text_gpt(comment_id, comment_text, argument_text)
            results.append(classification)

        except json.JSONDecodeError as e:
            print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
            continue

        except Exception as e:
            print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
            continue

    return results

classifications = process_comments_with_arguments(gm)

output_file = 'llm_argument_tasks/output_files/gpt4o-mini/comarg_gm_argument_identification.json'
with open(output_file, 'w') as f:
    json.dump(classifications, f, indent=2)

Processing comments: 100%|██████████| 1285/1285 [15:42<00:00,  1.36it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'llm_argument_tasks/output_files/gpt40-mini/comarg_gm_argument_identification.json'

In [49]:
output_file = 'comarg_gm_argument_identification.json'
with open(output_file, 'w') as f:
    json.dump(classifications, f, indent=2)

## Under God in Pledge

In [57]:
ugip = pd.read_csv('../../clean_data/UGIP_structured.csv')

def process_comments_with_arguments(df: pd.DataFrame) -> List[dict]:
    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing comments"):
        comment_id = row['id'] 
        comment_text = row['comment_text']  
        argument_text = row['argument_text']

        try:
            classification = classify_text_gpt(comment_id, comment_text, argument_text)
            results.append(classification)

        except json.JSONDecodeError as e:
            print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
            continue

        except Exception as e:
            print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
            continue

    return results

classifications = process_comments_with_arguments(ugip)

output_file = 'comarg_ugip_argument_identification_gpt.json'
with open(output_file, 'w') as f:
    json.dump(classifications, f, indent=2)

Processing comments:   0%|          | 0/1013 [00:00<?, ?it/s]

Processing comments: 100%|██████████| 1013/1013 [12:57<00:00,  1.30it/s]


## YRU

In [ ]:
def classify_text(id: str, comment_text: str, topic: str, label: str) -> dict:
    label_to_argument = topic_label_to_argument.get(topic)
    argument = label_to_argument.get(label)

    extract = client.chat.completions.create(
        messages=[
            {"role": "system", "content": f"""
            Analyze the given comment in relation to a specific argument about {topic}. You need to:
            Identify if the comment makes use of the given argument. If it does, assign the label 1. If it does not, assign the label 0.
            Do NOT use any other label.
            Do NOT include the comment or the argument in the response.
            
            The argument to analyze is: {argument}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "label": "the label for the use of the argument in the comment"
            }}
            
            Analyze the following comment in relation to the given argument:
            """},
            {"role": "user", "content": comment_text},
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": ArgumentClassification.model_json_schema(),
        }
    )
    
    return json.loads(extract.choices[0].message.content)

### Abortion

In [ ]:
def evaluate_predictions(golden_csv_path: str, predictions_json_path: str):
    """
    Evaluates predictions against the golden data.
    
    Parameters:
    - golden_csv_path: Path to the CSV file containing golden data.
    - predictions_json_path: Path to the JSON file containing predictions.
    
    Output:
    - Classification report comparing the predictions with the mapped gold labels.
    """

    golden_df = pd.read_csv(golden_csv_path)
    
    # Change labels to 1 for YRU data
    golden_df['label'] = 1 

    with open(predictions_json_path, 'r') as f:
        predictions = json.load(f)

    predictions_df = pd.DataFrame(predictions)
    
    # Ensure the IDs align between the golden data and the predictions
    merged_df = pd.merge(golden_df[['id', 'label']], predictions_df[['id', 'label']], on='id', suffixes=('_gold', '_pred'))

    # Map the gold labels to 0 and 1
    merged_df['label_gold'] = map_gold_labels(merged_df['label_gold'].astype(int))
    
    # Ground truth vector
    ground_truth = merged_df['label_gold'].values
    
    # Prediction vector
    predictions = merged_df['label_pred'].values

    report = classification_report(ground_truth, predictions)
    print(report)


## Evaluation GM

In [ ]:
golden_csv_path = '/Users/guida/llm_argument_tasks/clean_data/GM_structured.csv'
predictions_json_path = '/Users/guida/llm_argument_tasks/output_files/llama3/comarg_gm_argument_identification.json'

evaluate_predictions(golden_csv_path, predictions_json_path)

              precision    recall  f1-score   support

           0       0.77      0.82      0.80       836
           1       0.61      0.52      0.56       431

    accuracy                           0.72      1267
   macro avg       0.69      0.67      0.68      1267
weighted avg       0.71      0.72      0.72      1267

